<a href="https://colab.research.google.com/github/Thanutchaporn13/Test_course/blob/main/Parsing_quantity_of_Prime_and_Grade_on_product_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parsing quantity of Prime and Grade on product trial
Last updated: 2021-07-04
*   File path: G:\Publics\Operations_PP\Production_PP\Quality





## Import required Python modules and library

In [1]:
#Getting the required Python modules
!pip install xlrd
!pip install xlwt
!pip install openpyxl
!pip install xlsxwriter

     |████████████████████████████████| 153kB 4.1MB/s 


In [2]:
#Import library
import numpy as np
import pandas as pd
import os

## Upload the EXCEL file

In [17]:
from google.colab import files
uploaded = files.upload()

Saving Quality Report LN3_ Jan 21.xlsx to Quality Report LN3_ Jan 21.xlsx


In [201]:
# Getting file name after being loaded into Google Colab
filename = next(iter(uploaded))
print(filename)

## Getting line from filename
chunks = list(filename.split(' '))      
del chunks[3:5]
del chunks[0:2]
AA = [s.strip('_') for s in chunks]
List_line = list(AA)

Quality Report LN3_ Jan 21.xlsx


In [202]:
# move the io call to another variable
import io
df = pd.read_excel(io.BytesIO(uploaded.get(filename)))
## Move the io call to another variable
xlsx_file = io.BytesIO(uploaded.get(filename))

In [239]:
# Specify the sheet name
if List_line == ['LN1']:
  sheet_name01 = 'Grade HMC-1'
  sheet_name02 = 'LN1 Development product'
  List_line = ['PP1']
elif List_line == ['LN2']:
    sheet_name01 = 'Grade HMC-2'
    sheet_name02 = 'LN2 Development product'
    List_line = ['PP2']
elif List_line == ['LN3']:
      sheet_name01 = 'Grade HMC-3'
      sheet_name02 = 'LN3 Development product'
      List_line = ['PP3']

## Make dataframe for sheet 'Grade HMC-X'
df_sheet_grade = pd.read_excel(xlsx_file, sheet_name01)

## Make dataframe for sheet 'LNX Development product'
df_sheet_DevelopProduct = pd.read_excel(xlsx_file, sheet_name02)

## Retrieve "Prime" data from sheet: LNX Development product

In [204]:
# Make the 2nd row as header
new_header = df_sheet_DevelopProduct.iloc[1]
df_sheet_DevelopProduct = df_sheet_DevelopProduct[2:22]
df_sheet_DevelopProduct.columns = new_header

# Delete first-two columns
df_sheet_DevelopProduct = df_sheet_DevelopProduct.iloc[:,2:]

# Delete NaN column
df_sheet_DevelopProduct = df_sheet_DevelopProduct.dropna(axis=1,how='all')

# Delete Reason column
df_sheet_DevelopProduct = df_sheet_DevelopProduct.drop([5, 9, 13, 17, 21], axis=0)

# Transpose index and columns of the data frame.
df_DevelopProduct = df_sheet_DevelopProduct.T

# Set a new column name
df_DevelopProduct.columns = ['Product','Lot NO.', 'Prime']*5

In [205]:
# Rearrange data
## Subset data
df_DevelopProduct1 = df_DevelopProduct.iloc[:,:3]
df_DevelopProduct2 = df_DevelopProduct.iloc[:,3:6]
df_DevelopProduct3 = df_DevelopProduct.iloc[:,6:9]
df_DevelopProduct4 = df_DevelopProduct.iloc[:,9:12]
df_DevelopProduct5 = df_DevelopProduct.iloc[:,12:15]

## Append data
df_prime = df_DevelopProduct1.append([df_DevelopProduct2, df_DevelopProduct3, df_DevelopProduct4, df_DevelopProduct5])

## Delete NaN data and G-1
df_prime = df_prime.dropna()

## Delete rows of G-1
df_prime = df_prime[df_prime['Product'].str.contains("G-1")==False]
print(df_prime)

     Product   Lot NO.    Prime
1                              
7.0   HP500L  60108299  281.832
11.0  RC2472  60108307   69.925
12.0  RC2472  60108310   74.002
13.0  RC2472  60108418  164.412
7.0   HP500L  60108300  281.832
9.0   RP171G  60108323   280.76
10.0  RC2472  60108305  280.011
11.0  RC2472  60108308   280.04
12.0  RC2472  60108311  282.002
13.0  RC2472  60108238   97.455
7.0   HP500L  60108301  281.832
10.0  RC2472  60108306  281.678
11.0  RC2472  60108309  280.931
12.0  RC2472  60108417  280.929
10.0  RC2472  60108307      211
11.0  RC2472  60108310      208
12.0  RC2472  60108418  150.831


## Retrieve "Grade" data from sheet: Grade HMC-X

In [240]:
# Delete the first-two rows
df_sheet_grade = df_sheet_grade.drop([0,1],axis=0)
# Delete unnecessary columns
df_sheet_grade = df_sheet_grade.drop(df_sheet_grade.loc[:,'Sum Grade':'Unnamed: 34'].columns, axis=1)
### Change column name
df_sheet_grade.columns = ['Date','Cause','Type','OFF-Cause','From','To','Grade','Forecast', 'Note']

,Date,Cause,Type,OFF-Cause,From,To,Grade,Forecast,Note
2,2021-01-01 00:00:00,NaN,NaN,NaN,HP525J,HP400K,0.0,0,NaN
3,2021-01-01 00:00:00,NaN,NaN,NaN,HP525J,HP400M,0.0,X,NaN
4,2021-01-01 00:00:00,NaN,NaN,NaN,HP400M,NaN,NaN,0,NaN
5,2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
6,2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
7,2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
8,2021-01-02 00:00:00,NaN,NaN,NaN,HP400M,HP520M,0.0,0,NaN
9,2021-01-02 00:00:00,NaN,NaN,NaN,HP520M,0,NaN,0,NaN
10,2021-01-02 00:00:00,NaN,NaN,NaN,0,NaN,NaN,0,NaN
11,2021-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [241]:
# Set day as index
## Extract Day and create a dedicated column from column ['Date']
df_sheet_grade['Date'] = pd.to_datetime(df_sheet_grade['Date'])
df_sheet_grade['Day'] = df_sheet_grade['Date'].dt.day

### Set Day as Index
df_sheet_grade.set_index('Day', inplace=True)

df_sheet_grade.head(5)

,Date,Cause,Type,OFF-Cause,From,To,Grade,Forecast,Note
Day,,,,,,,,,
1.0,2021-01-01,NaN,NaN,NaN,HP525J,HP400K,0.0,0,NaN
1.0,2021-01-01,NaN,NaN,NaN,HP525J,HP400M,0.0,X,NaN
1.0,2021-01-01,NaN,NaN,NaN,HP400M,NaN,NaN,0,NaN
1.0,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1.0,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


## Combine 'Prime' and 'Grade' data

In [242]:
# Create an empty DataFrame for storing data
columns = ['Month','Year', 'Line', 'Cause','Type','OFF-Cause','Product','From','To','Grade','Prime', 'Note','Lot NO.']
df_all = pd.DataFrame(columns=columns)

In [243]:
# Append df_prime first and add data in columns: 'From', 'To' and 'Type'
df_all = df_all.append(df_prime)
df_all['From'] = df_all['Product']
df_all['To'] = df_all['Product']

rows_all =  len(df_all.index)
list_type = ['Prime']*rows_all
df_all['Type'] = list_type
df_all.head(3)

,Month,Year,Line,Cause,Type,OFF-Cause,Product,From,To,Grade,Prime,Note,Lot NO.
7.0,NaN,NaN,NaN,NaN,Prime,NaN,HP500L,HP500L,HP500L,NaN,281.832,NaN,60108299
11.0,NaN,NaN,NaN,NaN,Prime,NaN,RC2472,RC2472,RC2472,NaN,69.925,NaN,60108307
12.0,NaN,NaN,NaN,NaN,Prime,NaN,RC2472,RC2472,RC2472,NaN,74.002,NaN,60108310


In [244]:
# Append df_Grade
list_index_prime = df_all.index.to_list()
list_index_prime = list(dict.fromkeys(list_index_prime))
df_Grade = df_sheet_grade.loc[list_index_prime,:]
df_Grade = df_Grade.dropna(subset=['Grade'])
df_all = df_all.append(df_Grade)
df_all = df_all.sort_index("index", ascending=True)
df_all = df_all.reset_index()
df_all = df_all.rename(columns={'index':'Day'})
df_all = df_all.drop('Date', axis=1)
df_all = df_all.dropna(subset=['Type'])
# Replace 0 in column ['To']
df_all['To'] = np.where(df_all['To']==0, df_all['From'], df_all['To'])
df_all.head(5)

,Day,Month,Year,Line,Cause,Type,OFF-Cause,Product,From,To,Grade,Prime,Note,Lot NO.,Forecast
0,7.0,NaN,NaN,NaN,NaN,Prime,NaN,HP500L,HP500L,HP500L,NaN,281.832,NaN,60108299,NaN
2,7.0,NaN,NaN,NaN,NaN,Prime,NaN,HP500L,HP500L,HP500L,NaN,281.832,NaN,60108300,NaN
3,7.0,NaN,NaN,NaN,NaN,Prime,NaN,HP500L,HP500L,HP500L,NaN,281.832,NaN,60108301,NaN
4,9.0,NaN,NaN,NaN,T,PTC,NaN,NaN,HP400K,RP171G,412.987,NaN,NaN,NaN,700
5,9.0,NaN,NaN,NaN,NaN,Prime,NaN,RP171G,RP171G,RP171G,NaN,280.76,NaN,60108323,NaN
6,10.0,NaN,NaN,NaN,NaN,Prime,NaN,RC2472,RC2472,RC2472,NaN,211,NaN,60108307,NaN
7,10.0,NaN,NaN,NaN,T,PTC,NaN,NaN,RP171G,RC2472,134.015,NaN,NaN,NaN,300
8,10.0,NaN,NaN,NaN,NaN,Prime,NaN,RC2472,RC2472,RC2472,NaN,281.678,NaN,60108306,NaN
9,10.0,NaN,NaN,NaN,NaN,Prime,NaN,RC2472,RC2472,RC2472,NaN,280.011,NaN,60108305,NaN
10,10.0,NaN,NaN,NaN,T,SC,NaN,NaN,RC2472,0,26.806,NaN,NaN,NaN,0


In [224]:
# Fill columns: Month, Year and Line
rows_all =  len(df_all.index)

iMonth = [df_sheet_grade.iat[0,0].month]
list_month = iMonth*rows_all
df_all['Month'] = list_month

iYear  = [df_sheet_grade.iat[0,0].year]
list_Year = iYear*rows_all
df_all['Year'] = list_Year


list_PPline = List_line*rows_all
df_all['Line'] = list_PPline

df_all

,Day,Month,Year,Line,Cause,Type,OFF-Cause,Product,From,To,Grade,Prime,Note,Lot NO.,Forecast
0,7.0,1,2021,PP3,NaN,Prime,NaN,HP500L,HP500L,HP500L,NaN,281.832,NaN,60108299,NaN
2,7.0,1,2021,PP3,NaN,Prime,NaN,HP500L,HP500L,HP500L,NaN,281.832,NaN,60108300,NaN
3,7.0,1,2021,PP3,NaN,Prime,NaN,HP500L,HP500L,HP500L,NaN,281.832,NaN,60108301,NaN
4,9.0,1,2021,PP3,T,PTC,NaN,NaN,HP400K,RP171G,412.987,NaN,NaN,NaN,700
5,9.0,1,2021,PP3,NaN,Prime,NaN,RP171G,RP171G,RP171G,NaN,280.76,NaN,60108323,NaN
6,10.0,1,2021,PP3,NaN,Prime,NaN,RC2472,RC2472,RC2472,NaN,211,NaN,60108307,NaN
7,10.0,1,2021,PP3,T,PTC,NaN,NaN,RP171G,RC2472,134.015,NaN,NaN,NaN,300
8,10.0,1,2021,PP3,NaN,Prime,NaN,RC2472,RC2472,RC2472,NaN,281.678,NaN,60108306,NaN
9,10.0,1,2021,PP3,NaN,Prime,NaN,RC2472,RC2472,RC2472,NaN,280.011,NaN,60108305,NaN
10,10.0,1,2021,PP3,T,SC,NaN,NaN,RC2472,0,26.806,NaN,NaN,NaN,0


# Save as csv file

In [225]:
# Set file name for saving as csv
savename = filename[:-5]+".csv"
print(savename)

# Exporting Data from google colab to local machine
from google.colab import files
df_all.to_csv(savename, sep = ';', index=False, header=True)
files.download(savename)

Quality Report LN3_ Jan 21.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>